In [1]:
def train_sent(read_f, write_f, length):
    with open(read_f, 'r') as f:
        count = 0
        for sent in f:
            if count >= length:
                break
            with open(write_f, 'a+') as w:
                if len(sent) > 1:
                    w.write(sent)
                    count += 1

In [2]:
def txt_write_csv(csv_file, txt_file):
    import pandas as pd
    sent = []
    label = []
    with open(txt_file, 'r') as f:
        for line in f:
            sent.append(line)
            label.append('n')
    dataframe = pd.DataFrame({'sentence':sent, 'label':label})
    dataframe.to_csv(csv_file, index=False, sep=',')

In [3]:
import pandas as pd
import numpy as np
from gensim.models import word2vec

def sent_vector(csv_file, length):
    csv_data = pd.read_csv(csv_file, low_memory = False, nrows=length)#防止弹出警告
    csv_df = pd.DataFrame(csv_data)
    sentence = csv_df['sentence']
    label = csv_df['label']
    label.loc[label == 'n'] = 0
    label.loc[label == 'p'] = 1
    model = word2vec.Word2Vec.load('Model/n_cbow.w2v')
    size = model.wv.syn0.shape[1]
    sents_vec = []
    for sent in sentence:
        sents_vec.append(compute_sent_vec(sent, model, size))
    return np.array(sents_vec), label
        

In [4]:
def compute_sent_vec(sent, model, size):
    vec = np.zeros(size)
    for word in sent.split():
        if word in model.wv:
            vec += model.wv[word]
    vec /= len(sent.split())
    return vec


In [5]:
sents_vec, label = sent_vector('Data/training_data.csv', 1000)

/opt/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  del sys.path[0]


In [17]:
#self implemented logistic regression
from logistic_regression import LogisticRegression
l = LogisticRegression(sents_vec[:700], label[:700])
cost = l.train(0.1,1000)
pred_y = l.predict(sents_vec[300:], label[300:])

cost value is 0.6931471805599452
cost value is 0.4554618528642027
cost value is 0.42227252767043266
cost value is 0.4061907925545195
cost value is 0.3962050024671608
cost value is 0.3893107383547077
cost value is 0.3842631264306109
cost value is 0.3804264676315368
cost value is 0.37743241093104785
cost value is 0.3750489256740978
正确率为81.5%


In [7]:
#sklearn logistic regression result
from sklearn.linear_model import LogisticRegression
label = np.array(label, dtype='int')
clf = LogisticRegression(max_iter=1000).fit(sents_vec[:700], label[:700])
test_accuracy = clf.score(sents_vec[300:], label[300:])*100
pred = clf.predict(sents_vec[300:])
print("正确率为%s%%" % test_accuracy)

正确率为79.57142857142857%


In [8]:
pred

array([0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,

In [9]:
pred_y

array([0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,